In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

In [ ]:
df = pd.read_csv('./raw/ShenzhenTrips.csv', sep=',', index_col=0)

In [ ]:
df

In [ ]:
df = df[['Lon', 'Lat', 'GPSTime', 'TrajNumber']]

In [ ]:
def test1(traj_list, global_feature):
    n = len(traj_list)
    edge_weights_matrix = np.sum(np.minimum(global_feature[:n-1, np.newaxis], 
                                            global_feature[np.arange(1, n)][:, np.newaxis]), 
                                axis=2)

    # 获取所有非零的edge_weights以及它们的indices
    non_zero_indices = np.argwhere(edge_weights_matrix)

    global_edge_list = [[idx[0], idx[0]+idx[1]+1, edge_weights_matrix[idx[0], idx[1]]] 
                        for idx in non_zero_indices]

    edge_weight_max = edge_weights_matrix.max()
    return global_edge_list, edge_weight_max

In [ ]:
def test1(traj_list, global_feature):
    n = len(traj_list)
    edge_weights_matrix = np.sum(np.minimum(global_feature[:n-1, np.newaxis], 
                                            global_feature[np.arange(1, n)][:, np.newaxis]), 
                                axis=2)

    # 获取所有非零的edge_weights以及它们的indices
    non_zero_indices = np.argwhere(edge_weights_matrix)

    global_edge_list = [[idx[0], idx[0]+idx[1]+1, edge_weights_matrix[idx[0], idx[1]]] 
                        for idx in non_zero_indices]

    edge_weight_max = edge_weights_matrix.max()
    return global_edge_list, edge_weight_max
def test2(traj_list, global_feature):
    global_edge_list, edge_weight_max = [], 0
    for node1 in tqdm(range(len(traj_list)-1)):
            node2_list = [idx for idx in range(node1+1, len(traj_list))]
            edge_weight_list = np.sum(np.minimum(
                global_feature[node1], global_feature[node2_list]), axis=1)
            edge_weight_max = max(edge_weight_list.max(), edge_weight_max)
            for node2, edge_weight in zip(node2_list, edge_weight_list):
                if edge_weight:
                    global_edge_list.append([node1, node2, edge_weight])
    return global_edge_list, edge_weight_max

# def test3(traj_list, global_feature):
#     n = len(traj_list)
#     global_edge_list = []
#     for node1 in tqdm(range(n-1)):
#         node2_start = node1 + 1
#         edge_weights = np.sum(np.minimum(global_feature[node1], global_feature[node2_start:]), axis=1)
#         non_zero_indices = np.where(edge_weights > 0)[0]

#         for idx in non_zero_indices:
#             global_edge_list.append([node1, idx + node2_start, edge_weights[idx]])

#     edge_weight_max = np.max([edge[2] for edge in global_edge_list])
#     return global_edge_list, edge_weight_max

def test4(traj_list, global_feature):
    n = len(traj_list)
    edge_weight_max = float('-inf')  # 初始值设为负无穷大

    # 用来存储边的列表
    global_edge_list = []

    for node1 in tqdm(range(n - 1)):
        # 使用numpy的广播功能直接计算
        min_weights = np.minimum(global_feature[node1], global_feature[node1+1:])
        edge_weights = np.sum(min_weights, axis=1)

        # 更新edge_weight_max
        current_max = edge_weights.max()
        if current_max > edge_weight_max:
            edge_weight_max = current_max

        # 找到非零权重的边并添加到列表中
        nonzero_indices = np.nonzero(edge_weights)[0]
        for idx in nonzero_indices:
            node2 = node1 + 1 + idx
            global_edge_list.append([node1, node2, edge_weights[idx]])
    return global_edge_list, edge_weight_max


In [ ]:
global_edge_list1, edge_weight_max1 = test1(traj_list, global_feature)
# global_edge_list2, edge_weight_max2 = test2(traj_list, global_feature)
# global_edge_list3, edge_weight_max3 = test3(traj_list, global_feature)
global_edge_list4, edge_weight_max4 = test4(traj_list, global_feature)

In [2]:
def convert_time_format(time_str):
    dt = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")
    # dt = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%SZ")
    # return dt.strftime("%Y-%m-%d %H:%M:%S")

In [5]:
def convert_csv_to_dyna(csv_path):
    # 读取csv文件
    df = pd.read_csv('./raw/ShenzhenTrips.csv', sep=',', index_col=0)
    
    # 创建新的DataFrame列
    df['dyna_id'] = df.index
    df['type'] = 'trajectory'
    df['time'] = convert_time_format(str(df['GPSTime']))
    df['entity_id'] = df['ObjectID']
    df['traj_id'] = df['TrajNumber']
    df['coordinates'] = df.apply(lambda row: f'"[{row["Lat"]},{row["Lon"]}]"', axis=1)
    df['current_dis'] = 0.0  # 用固定数字填充
    df['current_state'] = 1.0  # 用固定数字填充
    
    # 选择并排序列
    dyna_df = df[['dyna_id', 'type', 'time', 'entity_id', 'traj_id', 'coordinates', 'current_dis', 'current_state']]
    
    # 将DataFrame转换为字符串格式
    dyna_content = dyna_df.to_csv(index=False, line_terminator='\n')
    
    return dyna_content

In [7]:
csv_path = './raw/ShenzhenTrips.csv'
# dyna_content = convert_csv_to_dyna(csv_path)

In [ ]:
# 读取csv文件
df = pd.read_csv('./raw/ShenzhenTrips.csv', sep=',', index_col=0)

# 创建新的DataFrame列
df['dyna_id'] = df.index
df['type'] = 'trajectory'
df['time'] = convert_time_format(str(df['GPSTime']))
df['entity_id'] = df['ObjectID']
df['traj_id'] = df['TrajNumber']
df['coordinates'] = df.apply(lambda row: f'"[{row["Lat"]},{row["Lon"]}]"', axis=1)
df['current_dis'] = 0.0  # 用固定数字填充
df['current_state'] = 1.0  # 用固定数字填充

# 选择并排序列
dyna_df = df[['dyna_id', 'type', 'time', 'entity_id', 'traj_id', 'coordinates', 'current_dis', 'current_state']]

# 将DataFrame转换为字符串格式
dyna_content = dyna_df.to_csv(index=False, line_terminator='\n')

In [8]:
def convert_time_format(time_str):
    # 将时间字符串解析为datetime对象
    dt = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
    
    # 将datetime对象格式化为.dyna文件的格式
    return dt.strftime("%Y-%m-%dT%H:%M:%SZ")

def convert_csv_to_dyna_updated(csv_path):
    # 读取csv文件
    df = pd.read_csv(csv_path)
    
    # 创建新的DataFrame列
    df['dyna_id'] = df.index
    df['type'] = 'trajectory'
    df['time'] = df['GPSTime'].apply(convert_time_format)
    df['entity_id'] = df['ObjectID']
    df['traj_id'] = df['TrajNumber']
    df['coordinates'] = df.apply(lambda row: f'[{row["Lat"]},{row["Lon"]}]', axis=1)
    df['current_dis'] = 0.0  # 用固定数字填充
    df['current_state'] = 1.0  # 用固定数字填充
    
    # 选择并排序列
    dyna_df = df[['dyna_id', 'type', 'time', 'entity_id', 'traj_id', 'coordinates', 'current_dis', 'current_state']]
    
    # 将DataFrame转换为字符串格式
    dyna_content = dyna_df.to_csv(index=False, line_terminator='\n')
    
    return dyna_content

In [11]:
csv_path = './raw/ShenzhenTrips.csv'



# 读取csv文件
df = pd.read_csv(csv_path)


In [12]:
df

,Unnamed: 0,Lon,Lat,Direct,GPSTime,TrajNumber
0,0,114.108773,22.537715,260,2016-01-01 12:07:25,1
1,1,114.107440,22.537368,250,2016-01-01 12:07:32,1
2,2,114.105513,22.537422,290,2016-01-01 12:07:52,1
3,3,114.104855,22.537737,290,2016-01-01 12:08:07,1
4,4,114.102812,22.538258,270,2016-01-01 12:08:22,1
...,...,...,...,...,...,...
656110,656110,114.094551,22.561838,178,2016-01-15 13:14:57,19542
656111,656111,114.095662,22.561496,93,2016-01-15 13:16:09,19542
656112,656112,114.096533,22.561444,92,2016-01-15 13:16:21,19542
656113,656113,114.097280,22.561424,94,2016-01-15 13:16:28,19542


In [ ]:

# 创建新的DataFrame列
df['dyna_id'] = df.index
df['type'] = 'trajectory'
df['time'] = df['GPSTime'].apply(convert_time_format)
df['entity_id'] = df['ObjectID']
df['traj_id'] = df['TrajNumber']
df['coordinates'] = df.apply(lambda row: f'[{row["Lat"]},{row["Lon"]}]', axis=1)
df['current_dis'] = 0.0  # 用固定数字填充
df['current_state'] = 1.0  # 用固定数字填充

# 选择并排序列
dyna_df = df[['dyna_id', 'type', 'time', 'entity_id', 'traj_id', 'coordinates', 'current_dis', 'current_state']]

# 将DataFrame转换为字符串格式
dyna_content = dyna_df.to_csv(index=False, line_terminator='\n')
